In [1]:
%reload_ext autoreload
%autoreload 2

import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [3]:
from app.system import System
from app.tools import PropertiesFilterTool, InfoExtractorTool
from app.system import Conversation, Task, Step
from app.task_resolver import GatherBookingInfoResolver, HouseSelectionResolver, GatherUserInfoResolver, BookingConfirmationResolver
from app.system import System
from app.task_resolver.tasks import create_make_reservation_task, create_task_router_task

In [4]:
# from app.system import Conversation, Task, Step
# from app.task_resolver import GatherBookingInfoResolver
# from app.system import System

# system_x = System()
# step = Step("GATHER_BOOKING_INFO", GatherBookingInfoResolver())
# customer_number="99513718"

# def test_step(message, customer_number="99513718"):
#     system_x.add_user_message(message, customer_number)
#     conversation = system_x.get_conversation(customer_number)
#     print(f"Conversation: {conversation}")
#     result = step.resolve(conversation)

#     print(f"Result: {result} \nStep Done: {step.is_done()}")
#     if not step.is_done():
#         system_x.add_assistant_message(result, customer_number)
#     else:
#         print("Step is Done, we can move to the next step.")

# test_step("2 personas para el sabado por 2 noches")

In [1]:
from app.system import Conversation, Task, Step
from app.task_resolver import GatherBookingInfoResolver, HouseSelectionResolver, GatherUserInfoResolver, BookingConfirmationResolver
from app.system import System
from app.task_resolver.tasks import create_make_reservation_task, create_task_router_task
import langchain

system = System()
customer_number="99513718"



def test_task(message: str, task: Task): 
    conversation = system.get_conversation(customer_number)
    conversation.add_user_message(message)
    print(f"Conversation: {conversation.get_messages()}")
    response = task.run(conversation.get_messages())
    conversation.add_assistant_message(response)
    system.save_conversation(conversation)
    return response

In [3]:
task = create_task_router_task()
# make_reservation_task = create_make_reservation_task()

In [4]:
#test_task("Hola", create_task_router_task())

test_task("Me gustaria reservar una casa", task)

Conversation: [{'role': 'user', 'content': 'Hola'}, {'role': 'assistant', 'content': 'Hola, ¿en qué puedo ayudarte con respecto a la reserva de una casa para estadías cortas?'}, {'role': 'user', 'content': 'Me gustaria reservar una casa'}]
Resolving Step: PROMPT_INJECTION_STEP
Resolving Step: EXIT_TASK_STEP
Resolving Step: TASK_IDENTIFIER_STEP


> Entering new  chain...
Prompt after formatting:
You are an Assistant that helps users perform actions related to booking a house for short-term stay.
Your task is to identify which action does the user want to do. You allways answer in Spanish.

Here is the list of possible actions:

MAKE_RESERVATION: the user wants to book an accomodation.
ASK_FOR_INFO: the user needs information about the properties available.
RESERVATION_INFORMATION: the user wants information about an accomodation booked by him before.
OTHER: when None of the actions described above fits.

Here is the conversation: 
user: Hola
assistant: Hola, ¿en qué puedo ayudarte con r

'¡Genial! ¿En qué fechas te gustaría reservar la casa y para cuántas personas?'

In [1]:
from app.system import System
from app.tools import PropertiesFilterTool, InfoExtractorTool
from app.system import Conversation, Task, Step
from app.task_resolver import GatherBookingInfoResolver, HouseSelectionResolver, GatherUserInfoResolver, BookingConfirmationResolver
from app.system import System
from app.task_resolver.tasks import create_make_reservation_task, create_task_router_task

system = System()

def create_task(task_name):
    if task_name == "MAKE_RESERVATION_TASK":
        return create_make_reservation_task()
    else:
        return create_task_router_task()

def main_flow(message: str, customer_number: str): 

    if message == "exit":
        system.save_conversation(Conversation(customer_number, create_task_router_task()))
        return "Conversacion reiniciada. Puedes comenzar de nuevo!"

    conversation = system.get_conversation(customer_number)
    conversation.add_user_message(message)
    print(f"Conversation: {conversation.get_messages()}")
    task_result = conversation.task.run(conversation.get_messages())
    if conversation.task.name == "TASK_ROUTER_TASK":
        if task_result["task_id"] != "OTHER":
            # Here I know already that he wants to make a reservation.
            task = create_task(task_result["task_id"])
            conversation.task = task
            response = task.run(conversation.get_messages())
            # if not task.steps[-1].reply_when_done:
            #     return main_flow()                
            conversation.add_assistant_message(response)
            system.save_conversation(conversation)
            return response
        else:
            conversation.add_assistant_message(task_result["text"])
            system.save_conversation(conversation)
            return task_result["text"]
    else:
        conversation.add_assistant_message(task_result)
        system.save_conversation(conversation)
        return task_result

In [2]:
main_flow("Hola", "12345")

Conversation: [{'role': 'user', 'content': 'Hola'}]
Resolving Step: PROMPT_INJECTION_STEP
Resolving Step: EXIT_TASK_STEP
Resolving Step: TASK_IDENTIFIER_STEP


> Entering new  chain...
Prompt after formatting:
You are an Assistant that helps users.
Your task is to identify which action does the user want to do. You allways answer in Spanish.

Here is the list of possible actions:

MAKE_RESERVATION_TASK: the user wants to book an accomodation.
PROPERTIES_INFORMATION_TASK: the user needs information about the properties available.
RESERVATION_INFORMATION_TASK: the user wants information about an accomodation booked before.
OTHER: when None of the actions described above fits.

Here is the conversation: 
user: Hola


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"task_id": string  // The task_id of the Task that the user wants to perform.
	"text": string  // The response to the user.
}
``

'Hola, ¿en qué puedo ayudarte?'

In [3]:
main_flow("Me gustaria reservar una casa para el finde", "12345")

Conversation: [{'role': 'user', 'content': 'Hola'}, {'role': 'assistant', 'content': 'Hola, ¿en qué puedo ayudarte?'}, {'role': 'user', 'content': 'Me gustaria reservar una casa para el finde'}]
Resolving Step: TASK_IDENTIFIER_STEP


> Entering new  chain...
Prompt after formatting:
You are an Assistant that helps users.
Your task is to identify which action does the user want to do. You allways answer in Spanish.

Here is the list of possible actions:

MAKE_RESERVATION_TASK: the user wants to book an accomodation.
PROPERTIES_INFORMATION_TASK: the user needs information about the properties available.
RESERVATION_INFORMATION_TASK: the user wants information about an accomodation booked before.
OTHER: when None of the actions described above fits.

Here is the conversation: 
user: Hola
assistant: Hola, ¿en qué puedo ayudarte?
user: Me gustaria reservar una casa para el finde


The output should be a markdown code snippet formatted in the following schema, including the leading and trail

'¡Hola! Claro, con gusto te ayudo a reservar una casa. Antes de empezar, ¿podrías proporcionarme la fecha de entrada que tienes en mente?'

In [4]:
main_flow("exit", "12345")

'Conversacion reiniciada. Puedes comenzar de nuevo!'

In [6]:
main_flow("Queria comprar huevos, cuanto estan?", "12345")

Conversation: [{'role': 'user', 'content': 'Hola de nuevo'}, {'role': 'assistant', 'content': 'Hola, ¿en qué puedo ayudarte?'}, {'role': 'user', 'content': 'Queria comprar huevos, cuanto estan?'}]
Resolving Step: TASK_IDENTIFIER_STEP


> Entering new  chain...
Prompt after formatting:
You are an Assistant that helps users.
Your task is to identify which action does the user want to do. You allways answer in Spanish.

Here is the list of possible actions:

MAKE_RESERVATION_TASK: the user wants to book an accomodation.
PROPERTIES_INFORMATION_TASK: the user needs information about the properties available.
RESERVATION_INFORMATION_TASK: the user wants information about an accomodation booked before.
OTHER: when None of the actions described above fits.

Here is the conversation: 
user: Hola de nuevo
assistant: Hola, ¿en qué puedo ayudarte?
user: Queria comprar huevos, cuanto estan?


The output should be a markdown code snippet formatted in the following schema, including the leading and t

'Lo siento, no puedo ayudarte con la compra de huevos. ¿Hay algo más en lo que pueda ayudarte?'